In [1]:
#import os
#os.system('GOOGLE_APPLICATION_CREDENTIALS="/home/user/Downloads/service-account-file.json"')

In [17]:
import pandas as pd
import numpy as np
import vaex

import glob
import itertools as itr
import collections as coll

import time

import sqlalchemy as sqla
import types
import json

In [18]:
%%capture
from tqdm.notebook import trange, tqdm
tqdm.pandas()

In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
# Replace the first with the second category
CATEGORY_ALIASES = {
    'math.MP': 'math-ph',
    'stat.TH': 'math.ST',
    'math.IT': 'cs.IT',
    'q-fin.EC': 'econ.GN',
    'cs.SY': 'eess.SY',
    'cs.NA': 'math.NA'
}

# Used subject categories, with aliases removed.
ACTIVE_CATEGORIES = [
    'astro-ph.CO','astro-ph.EP','astro-ph.GA',
    'astro-ph.HE','astro-ph.IM','astro-ph.SR',
    'cond-mat.dis-nn','cond-mat.mes-hall','cond-mat.mtrl-sci',
    'cond-mat.other','cond-mat.quant-gas','cond-mat.soft',
    'cond-mat.stat-mech','cond-mat.str-el','cond-mat.supr-con',
    'cs.AI','cs.AR','cs.CC','cs.CE','cs.CG','cs.CL','cs.CR',
    'cs.CV','cs.CY','cs.DB','cs.DC','cs.DL','cs.DM','cs.DS',
    'cs.ET','cs.FL','cs.GL','cs.GR','cs.GT','cs.HC','cs.IR',
    'cs.IT','cs.LG','cs.LO','cs.MA','cs.MM','cs.MS','cs.NE',
    'cs.NI','cs.OH','cs.OS','cs.PF','cs.PL','cs.RO','cs.SC',
    'cs.SD','cs.SE','cs.SI',
    'econ.EM','econ.GN','econ.TH',
    'eess.AS','eess.IV','eess.SP','eess.SY',
    'gr-qc',
    'hep-ex','hep-lat','hep-ph','hep-th',
    'math-ph','math.AC','math.AG','math.AP',
    'math.AT','math.CA','math.CO','math.CT',
    'math.CV','math.DG','math.DS','math.FA',
    'math.GM','math.GN','math.GR','math.GT',
    'math.HO','math.KT','math.LO','math.MG',
    'math.NA','math.NT','math.OA','math.OC',
    'math.PR','math.QA','math.RA','math.RT',
    'math.SG','math.SP','math.ST',
    'nlin.AO','nlin.CD','nlin.CG','nlin.PS','nlin.SI',
    'nucl-ex','nucl-th',
    'physics.acc-ph','physics.ao-ph','physics.app-ph',
    'physics.atm-clus','physics.atom-ph','physics.bio-ph',
    'physics.chem-ph','physics.class-ph','physics.comp-ph',
    'physics.data-an','physics.ed-ph','physics.flu-dyn',
    'physics.gen-ph','physics.geo-ph','physics.hist-ph',
    'physics.ins-det','physics.med-ph','physics.optics',
    'physics.plasm-ph','physics.pop-ph','physics.soc-ph',
    'physics.space-ph',
    'q-bio.BM','q-bio.CB','q-bio.GN','q-bio.MN','q-bio.NC',
    'q-bio.OT','q-bio.PE','q-bio.QM','q-bio.SC','q-bio.TO',
    'q-fin.CP','q-fin.GN','q-fin.MF','q-fin.PM',
    'q-fin.PR','q-fin.RM','q-fin.ST','q-fin.TR',
    'quant-ph',
    'stat.AP','stat.CO','stat.ME','stat.ML','stat.OT',
    'dg-ga', 'astro-ph'
]

CAT_REMAPPER = {x:x for x in ACTIVE_CATEGORIES} | CATEGORY_ALIASES


len(CATEGORY_ALIASES)
len(ACTIVE_CATEGORIES)
len(CAT_REMAPPER)



6

151

157

In [23]:
train_df = vaex.open("./data/fulltext/train_major_cats.hdf5")
test_df = vaex.open("./data/fulltext/test_major_cats.hdf5")

In [24]:
train_df.shape

(31786, 9)

In [7]:
vaex.settings.display.max_rows = 200
cat_count_df = train_df.groupby(['major_category', 'prime_category'], agg = vaex.agg.count('paper_id')).sort(['paper_id_count'])
cat_count_df
train_df.groupby('major_category', agg = vaex.agg.count('paper_id')).sort('paper_id_count')
vaex.settings.display.max_rows = 30

#,major_category,prime_category,paper_id_count
0,cs,cs.MA,42
1,cs,cs.SI,42
2,cs,cs.DS,42
3,cs,cs.PL,42
4,cs,cs.HC,42
5,cs,cs.SE,42
6,cs,cs.CL,42
7,cs,cs.GR,42
8,cs,cs.CE,42
9,cs,cs.NE,42


#,major_category,paper_id_count
0,dg-ga,446
1,econ,934
2,stat,1585
3,nlin,1594
4,cs,1596
5,gr-qc,1600
6,nucl-th,1600
7,q-fin,1600
8,eess,1600
9,quant-ph,1600


In [8]:
train_df.paper_id

Expression = paper_id
Length: 31,786 dtype: string (column)
-------------------------------------
    0      1110.1547
    1      1107.3567
    2     1610.06216
    3      1309.3106
    4      1410.2451
        ...         
31781  dg-ga/9712021
31782  dg-ga/9606008
31783  dg-ga/9506009
31784  dg-ga/9612013
31785  dg-ga/9707021

In [9]:
paper = dict(zip(train_df.column_names, train_df[0]))
paper

{'paper_id': '1110.1547',
 'version': '1',
 'yymm': '1110',
 'created': Timestamp('2011-10-07 14:37:25'),
 'title': 'The impact of CCD radiation damage on Gaia astrometry: I. Image location estimation in the presence of radiation damage',
 'abs_categories': 'astro-ph.IM',
 'abstract': 'The Gaia mission has been designed to perform absolute astrometric measurements with unprecedented accuracy; the end-of-mission parallax standard error is required to be 30 micro-arcseconds for a G2V type star of magnitude 15. These requirements set a stringent constraint on the accuracy of the estimation of the location of the stellar image on the CCD for each observation: e.g., 0.3 milli-arseconds (mas) or 0.005 pixels for the same V=15 G2V star. However the Gaia CCDs will suffer from charge transfer inefficiency (CTI) caused by radiation damage that will degrade the stellar image quality and may degrade the astrometric performance of Gaia if not properly addressed. For the first time at this level of 

In [21]:
from google.cloud import storage as gs
from google.cloud import exceptions as gerr

In [22]:
key_file_path = "/home/cjc73/.credentials/arxiv-production-e9574603e738.json"
client = gs.Client.from_service_account_json(key_file_path)

def get_last_version(paper):
    prefix_path = '/'.join([
        'txt',
        paper['major_category'] if '/' in paper['paper_id'] else 'arxiv', 
        paper['yymm'],
        paper['paper_id'].split('/')[-1],
        #"1001",
        #paper['yymm'],
        #paper['']
    ])
    blobs = client.list_blobs(
        'arxiv-production-data', 
        prefix=prefix_path
    )
    return max(blobs, key=lambda x: x.name) #latest version

Need to extract the components from record to build a path and test with different permissions. 

    

In [12]:
blob.path
blob.bucket
blob.name

NameError: name 'blob' is not defined

In [13]:
train_df.shape[0]

31786

In [14]:
with open('data/fulltext/train_major_cats_full.json', 'w') as outfile, \
    tqdm(total=train_df.shape[0]) as err_bar:
    
    fail_count = 0
    fail_idx = []
    decode_fail_count = 0
    decode_fail_idx = []

    for i, row in tqdm(enumerate(train_df.iterrows()), total=train_df.shape[0]):
        #print(row)
        paper = row[1]
        try:
            blob = get_last_version(paper)
            textbytes = blob.download_as_bytes()
        except (ValueError, gerr.GoogleCloudError):
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
            continue
            
        except gerr.GoogleCloudError:
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
        try: 
            text_string = textbytes.decode('utf-8')
            paper['fulltext'] = text_string  
        except UnicodeDecodeError:
            try:
                text_string = textbytes.decode("windows-1252")
                paper['fulltext'] = text_string  
            except:
                try:
                    text_string = textbytes.decode("cp1251")
                    paper['fulltext'] = text_string  
                except:
                    decode_fail_count += 1
                    decode_fail_idx.append(row[0])
                    _ = err_bar.update(err_bar.n + 1)

        paper['created'] = paper['created'].isoformat()
        _ = outfile.write(
                json.dumps(
                    paper, 
                    ensure_ascii=False
                )
            )
        _ = outfile.write('\n')
        time.sleep(.3)

print(f"{fail_count} errors of {i+1} completed.")
print(f"{decode_fail_count} decode fails of {i+1} completed.")
    
        
        

  0%|          | 0/31786 [00:00<?, ?it/s]

  0%|          | 0/31786 [00:00<?, ?it/s]

33 errors of 31786 completed.
0 decode fails of 31786 completed.


In [30]:
with open('data/fulltext/test_major_cats_full.json', 'w') as outfile, \
    tqdm(total=test_df.shape[0]) as err_bar:
    
    fail_count = 0
    fail_idx = []
    decode_fail_count = 0
    decode_fail_idx = []

    for i, row in tqdm(enumerate(test_df.iterrows()), total=test_df.shape[0]):
        #print(row)
        paper = row[1]
        try:
            blob = get_last_version(paper)
            textbytes = blob.download_as_bytes()
        except (ValueError, gerr.GoogleCloudError):
            fail_count += 1
            fail_idx.append(row[0])
            _ = err_bar.update(err_bar.n + 1)
            continue

        try: 
            text_string = textbytes.decode('utf-8')
            paper['fulltext'] = text_string  
        except UnicodeDecodeError:
            try:
                text_string = textbytes.decode("windows-1252")
                paper['fulltext'] = text_string  
            except:
                try:
                    text_string = textbytes.decode("cp1251")
                    paper['fulltext'] = text_string  
                except:
                    decode_fail_count += 1
                    decode_fail_idx.append(row[0])
                    _ = err_bar.update(err_bar.n + 1)

        paper['created'] = paper['created'].isoformat()
        _ = outfile.write(
                json.dumps(
                    paper, 
                    ensure_ascii=False
                )
            )
        _ = outfile.write('\n')
        time.sleep(.3)

print(f"{fail_count} errors of {i+1} completed.")
print(f"{decode_fail_count} decode fails of {i+1} completed.")
    
        
        

  0%|          | 0/7938 [00:00<?, ?it/s]

  0%|          | 0/7938 [00:00<?, ?it/s]

7 errors of 7938 completed.
0 decode fails of 7938 completed.


In [33]:
decode_fail_count
for x in decode_fail_idx:
    print(x)

0

In [34]:
for x in fail_idx:
    print(x)

397
3083
3402
4270
5439
7006
7934


In [131]:
paper = dict(zip(train_df.column_names, test_df[3783]))
paper

{'paper_id': '1605.03406',
 'version': '2',
 'yymm': '1605',
 'created': Timestamp('2016-11-29 08:34:34'),
 'title': 'Preface to the 18th workshop "What comes beyond the standard models", Bled July 11--19, 2015, and links to the talks in the proceedings',
 'abs_categories': 'hep-ph hep-ex hep-th',
 'abstract': 'The contribution contains the preface to the Proceedings to the 18th Workshop "What Comes Beyond the Standard Models", Bled, July 11 - 19, 2015, published in Bled workshops in physics, Vol.16, No. 2, DMFA-Zaloznistvo, Ljubljana, Dec. 2015, links to (most of) the published contributions and section (by M.Yu. Khlopov) on VIA at Bled 2015.',
 'prime_category': 'hep-ph',
 'major_category': 'hep-ph'}

In [132]:
blob = get_last_version(paper)
testbytes = blob.download_as_bytes()

ValueError: max() arg is an empty sequence

In [136]:
testbytes[230:330].decode("cp1251")

'\nUniv. Paris Diderot, Sorbonne Paris Citґ , 4 rue Elsa Morante, 75205 Paris Cedex e\n\n13\n3 4\n\nDepartm'

In [137]:
testbytes[230:330].decode("windows-1252")

'\nUniv. Paris Diderot, Sorbonne Paris Cit´ , 4 rue Elsa Morante, 75205 Paris Cedex e\n\n13\n3 4\n\nDepartm'

In [138]:
testbytes[230:330].decode("latin-1")

'\nUniv. Paris Diderot, Sorbonne Paris Cit´ , 4 rue Elsa Morante, 75205 Paris Cedex e\n\n13\n3 4\n\nDepartm'

In [120]:
testbytes[265:330].decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb4 in position 5: invalid start byte

In [80]:
fulltext[:200]

'Mon. Not. R. Astron. Soc. 000, 1 -- 21 (2011)\n\nPrinted 10 November 2018\n\n(MN LATEX style file v2.2)\n\nThe impact of CCD radiation damage on Gaia astrometry:\nI. Image location estimation in the presence'

In [118]:
b'\x98'.decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x98 in position 0: invalid start byte

In [72]:
sorted(blob_ls, key=lambda x: x.name)

[<Blob: arxiv-production-data, txt/arxiv/1410/1410.2451v1.txt, 1628900653232500>,
 <Blob: arxiv-production-data, txt/arxiv/1410/1410.2451v2.txt, 1628900653257189>]

In [73]:
min?

Docstring:
min(iterable, *[, default=obj, key=func]) -> value
min(arg1, arg2, *args, *[, key=func]) -> value

With a single iterable argument, return its smallest item. The
default keyword-only argument specifies an object to return if
the provided iterable is empty.
With two or more arguments, return the smallest argument.
Type:      builtin_function_or_method
